In [4]:
import requests
import time
import pandas as pd

In [5]:
#I kept API queries basic, just using the brand name of some of the top effects pedal vendors
queries = ['boss', 'electro harmonix', 'mxr', 'dunlop', 'earthquaker devices', 'tc electronic',
          'mooer', 'line 6', 'zvex', 'wampler', 'digitech', 'jhs', 'ibanez']

In [6]:
#The reverb API requires plus signs in between keywords. This has been updated with the list comprehension below
clean_queries = [query.replace(' ', '+') if ' ' in query else query for query in queries]

In [57]:
def reverb_get(query, product_type, per_page, page_limit=5):
    all_results = []
    endpoint = 'https://reverb.com/'
    params = {
        'product_type': product_type,
        'query': query,
        'per_page': per_page
    }
    first_request = requests.get(endpoint + 'api/listings/all', params=params).json()
    all_results.append(first_request)
    def get_next_page(next_page):
        if next_page:
            new_endpoint = endpoint + next_page
            new_r = requests.get(new_endpoint, params=params).json()
            all_results.append(new_r)
            if new_r.get('current_page') != page_limit:
                return get_next_page(new_r.get('_links').get('next').get('href'))
    get_next_page(first_request.get('_links').get('next').get('href'))
    return all_results

In [58]:
test = reverb_get(queries[0], 'effects-and-pedals', 50)

In [101]:
def parse_reverb_json(obj):
    all_listing_json = []
    listings = obj.get('listings')
    for l in listings:
        l_dict = {
            'category_uuids': l.get('category_uuids'),
            'condition': l.get('condition'),
            'created_at': l.get('created_at'),
            'description': l.get('description'),
            'finish': l.get('finish'),
            'has_inventory': l.get('has_inventory'),
            'id': l.get('id'),
            'inventory': l.get('inventory'),
            'listing_currency': l.get('listing_currency'),
            'make': l.get('make'),
            'model': l.get('model'),
            'offers_enabled': l.get('offers_enabled'),
            'price.amount': l.get('price').get('amount'),
            'price.currency': l.get('price').get('currency'),
            'price.display': l.get('price').get('display'),
            'price.symbol': l.get('price').get('symbol'),
            'published_at': l.get('published_at'),
            'shop_name': l.get('shop_name'),
            'title': l.get('title'),
            'year': l.get('year'),
            'web_url': l.get('_links').get('web').get('href')     
        }
        if l.get('shipping'):
            l_dict['shipping.us'] = l.get('shipping').get('us')
            l_dict['shipping.local'] = l.get('shipping').get('local')
            if l.get('shipping').get('us_rate'):
                l_dict['shipping.us_rate.amount'] = l.get('shipping').get('us_rate').get('currency')
                l_dict['shipping.us_rate.display'] = l.get('shipping').get('us_rate').get('display')
                l_dict['shipping.us_rate.symbol'] = l.get('shipping').get('us_rate').get('symbol')
        all_listing_json.append(l_dict)
    return all_listing_json

In [102]:
final_df = pd.concat([pd.DataFrame(parse_reverb_json(t)) for t in test], ignore_index=True).head()

In [104]:
final_df.head()

,category_uuids,condition,created_at,description,finish,has_inventory,id,inventory,listing_currency,make,...,published_at,shipping.local,shipping.us,shipping.us_rate.amount,shipping.us_rate.display,shipping.us_rate.symbol,shop_name,title,web_url,year
0,[66fd5c3b-3227-4182-9337-d0e4893be9a2],Brand New,2017-03-09T13:54:28-06:00,"<b>Boss BCB60 Pedal Case, It's a Case... It's ...",,True,4556240,17,USD,Boss,...,2017-03-09T14:18:11-06:00,False,True,USD,FREE,$,American Musical Supply,Boss BCB60 Pedal Case for up to 6 Boss Pedals,https://reverb.com/item/4556240-boss-bcb60-ped...,
1,[732e30f0-21cf-4960-a3d4-bb90c68081db],Good,2017-06-13T18:46:58-05:00,<p>\n\t<b>Boss MT-2: Metal Zone Guitar Distort...,,False,5640165,1,USD,Boss,...,2017-06-13T18:47:33-05:00,True,True,USD,$10,$,Haight Ashbury Music Center,Boss Used Boss MT-2: Metal Zone Guitar Distort...,https://reverb.com/item/5640165-boss-used-boss...,
2,"[3b09f948-3462-4ac2-93b3-59dd66da787e, 1738a9a...",Brand New,2013-04-24T20:02:13-05:00,<p>One of the most beloved echo effects ever m...,,True,5712,4,USD,Boss,...,2013-04-24T20:02:13-05:00,True,True,USD,FREE,$,Chicago Music Exchange,Boss RE-20 Space Echo,https://reverb.com/item/5712-boss-re-20-space-...,
3,[c6602a28-e2e7-4e70-abeb-0fa38b320be6],Brand New,2015-10-28T14:08:46-05:00,The BOSS SYB-3 was the world’s first bass synt...,,True,1269417,1,USD,Boss,...,2015-10-28T14:08:48-05:00,True,True,USD,FREE,$,Bananas at Large®,Boss SYB-5 Bass Synthesizer Pedal,https://reverb.com/item/1269417-boss-syb-5-bas...,
4,[2d6093b4-6b33-474e-b07c-25f6657d7956],Brand New,2016-12-09T15:57:31-06:00,"Get The New, Easy-To-Use Vocal Pedal That's B...",,True,3578196,1,USD,Boss,...,2016-12-16T07:04:06-06:00,False,True,USD,FREE,$,Chuck Levin's Washington Music Center,BOSS VE-20 Vocal Performer Effect Pedal,https://reverb.com/item/3578196-boss-ve-20-voc...,
